In [255]:
import csv
import json
import time
import heapq
from datetime import datetime, timedelta
from math import sqrt
import math
import random

In [256]:
def read_csv_to_matrix(file_path):
    data_matrix = []

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        for row in csv_reader:
            data_matrix.append(row)

    return data_matrix

csv_file_path = 'drivers.csv'
drivers_matrix = read_csv_to_matrix('drivers.csv')
passengers_matrix = read_csv_to_matrix('passengers.csv')
edges_matrix = read_csv_to_matrix('edges.csv')

In [ ]:
print("Drivers")
for i, row in enumerate(drivers_matrix[:5]):
    print(row)
print("Passengers")
for i, row in enumerate(passengers_matrix[:5]):
    print(row)
print("Edges")
for i, row in enumerate(edges_matrix[:5]):
    print(row)

In [257]:
def read_json_to_dict(file_path):
    with open(file_path, 'r') as json_file:
        data_dict = json.load(json_file)

    return data_dict

json_file_path = 'node_data.json'
node_data_dict = read_json_to_dict(json_file_path)

count = 0
for node_id, coordinates in node_data_dict.items():
    print(f'Node ID: {node_id}, Coordinates: {coordinates}')
    count += 1
    if count == 5:
        break

Node ID: 42467330, Coordinates: {'lon': -73.933676, 'lat': 40.655156}
Node ID: 42467331, Coordinates: {'lon': -73.932706, 'lat': 40.655216}
Node ID: 42467333, Coordinates: {'lon': -73.931772, 'lat': 40.655273}
Node ID: 42467334, Coordinates: {'lon': -73.930768, 'lat': 40.655336}
Node ID: 42467335, Coordinates: {'lon': -73.929762, 'lat': 40.655398}


# T1 
- Create quques to keep track of available drivers and passengers 
- When a driver is free, assign longest waiting passenger to them (the one with the earliest date/time)
- Disregard distances
- When matched, remove driver and passenger from the queue
- update to recycle drivers (find a way to estimate how long each ride takes-- use the graph?)

In [258]:
def match_passenger_to_driver(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        current_passenger_time, current_passenger = heapq.heappop(passenger_heap)
        # remove conditional, assign first passenger to first driver
        print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver(drivers_matrix[1:], passengers_matrix[1:])

Driver assigned to Passenger: ['04/25/2014 00:14:00', '40.667', '-73.8713'] -> ['04/25/2014 00:00:00', '40.6466', '-73.7896', '40.7603', '-73.9794']
Driver assigned to Passenger: ['04/25/2014 00:21:00', '40.6482', '-73.7831'] -> ['04/25/2014 00:07:00', '40.7155', '-73.9933', '40.7205', '-73.9872']
Driver assigned to Passenger: ['04/25/2014 00:31:00', '40.7648', '-73.996'] -> ['04/25/2014 00:08:00', '40.7157', '-74.0064', '40.7064', '-74.0167']
Driver assigned to Passenger: ['04/25/2014 00:14:00', '40.667', '-73.8713'] -> ['04/25/2014 00:10:00', '40.7523', '-73.9953', '40.7563', '-73.9915']
Driver assigned to Passenger: ['04/25/2014 00:37:00', '40.74', '-73.999'] -> ['04/25/2014 00:14:00', '40.7497', '-73.9952', '40.7455', '-73.9841']
Driver assigned to Passenger: ['04/25/2014 00:21:00', '40.6482', '-73.7831'] -> ['04/25/2014 00:20:00', '40.7298', '-74.0092', '40.7522', '-73.9795']
Driver assigned to Passenger: ['04/25/2014 00:44:00', '40.7396', '-74.0054'] -> ['04/25/2014 00:20:00', '4

How do we simulate that the ride is over for a driver passenger par so we can adsign that driver to a new passenger? This would be the case of T1, 2 and 3. 
- We were thinking we could assign an arbitrary time for each ride to take place, then add the driver back into the queue 
- Or we can use a heuristic to estimate time of the ride depending on the distance between the coordinates (the straight line distance)
- Or should we pre process the graph for all of the tasks? Have an accurate calculation of the time the ride takes to finish? 
- Do we even have to consider the ride finishing logic? 



In [ ]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Calculate the Euclidean distance between two sets of coordinates
    dx = lon2 - lon1
    dy = lat2 - lat1
    distance = sqrt(dx**2 + dy**2)
    return distance

def match_passenger_to_driver_t2(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    # heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        dist = float('inf')
        current_passenger = 0
        #count = -1
        #current_passenger_ind = 0
        for passenger in passenger_heap:
            #count = count + 1
            if passenger[0] < current_driver_time:
                if euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2])) < dist:
                    dist = euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2]))
                    current_passenger = passenger
                    #current_passenger_ind = count
        if current_passenger != 0:
            passenger_heap.remove(current_passenger)
        # remove conditional, assign first passenger to first driver
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger[1]}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver_t2(drivers_matrix[1:], passengers_matrix[1:])

## PreProcessing the Graph

creating a node dictionary, an edges dictionary, and an adjacency dictionary

In [259]:
def read_graph_nodes_and_edges():
    with open('node_data.json') as nodes_file:
        nodes = json.load(nodes_file)

    edges = {}
    with open('edges.csv', 'r') as edges_csv:
        edges_reader = csv.DictReader(edges_csv)
        for row in edges_reader:
            start_id = int(row['start_id'])
            end_id = int(row['end_id'])
            length = float(row['length'])

            # Additional attributes for each hour of the day
            weekday_hours = [float(row[f'weekday_{i}']) for i in range(24)]
            weekend_hours = [float(row[f'weekend_{i}']) for i in range(24)]

            edge = (start_id, end_id)

            edges[edge] = {
                'length': length,
                'weekday_hours': weekday_hours,
                'weekend_hours': weekend_hours
            }

    return nodes, edges
nodes, edges = read_graph_nodes_and_edges()

# Display the result
print("Graph Nodes:")

for node_id, node_data in list(nodes.items())[:5]:
    print(f'Node ID: {node_id}, Node Data: {node_data}')

print("\nGraph Edges:")
for edge, attributes in list(edges.items())[:5]:
    print(f'Edge: {edge}, Attributes: {attributes}')

Graph Nodes:
Node ID: 42467330, Node Data: {'lon': -73.933676, 'lat': 40.655156}
Node ID: 42467331, Node Data: {'lon': -73.932706, 'lat': 40.655216}
Node ID: 42467333, Node Data: {'lon': -73.931772, 'lat': 40.655273}
Node ID: 42467334, Node Data: {'lon': -73.930768, 'lat': 40.655336}
Node ID: 42467335, Node Data: {'lon': -73.929762, 'lat': 40.655398}

Graph Edges:
Edge: (39076461, 42854803), Attributes: {'length': 0.1582682216504902, 'weekday_hours': [19.8665, 19.986, 17.5154094776635, 23.146191909356634, 28.761, 22.698500000000003, 20.252, 15.733307692307692, 17.98772222222222, 20.90642857142857, 16.653230769230767, 15.671076923076924, 15.118333333333332, 14.84723076923077, 10.344733333333332, 6.3204, 12.116307692307691, 11.693833333333332, 13.2156, 16.782888888888888, 13.669571428571428, 15.879916666666666, 17.7299, 17.159000000000002], 'weekend_hours': [17.1085, 18.66775, 21.257666666666665, 21.91733333333333, 20.2505, 16.061, 21.858666666666664, 19.485, 15.53875, 19.924750000000003

In [269]:
# Create clusters graph 
def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 3)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)

def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 3))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 3))

        #print(cluster_key1,cluster_key2)
        # check if source and target are in different clusters aka they have different cluster keys 
        # if they are different, add it to the graph as an edge
        if cluster_key1 != cluster_key2:
            length = attributes["length"]
            speeds = attributes['weekday_hours']
            average_speed = sum(speeds)/len(speeds)
            estimated_time = length / average_speed
            # add this edge to the adjacency
            # Add source to target with weight
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(estimated_time)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [estimated_time]
        else:
            adjacency_dict[cluster_key1] = {cluster_key2: [estimated_time]}

        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency = update_edges_between_clusters(clusters, nodes)


for key1, nested_dict in adjacency.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            avg_list = sum(value) / len(value) 
            
            # Update the nested dictionary with the average list
            adjacency[key1][key2] = avg_list

print(len(adjacency))
for key, value in list(adjacency.items())[:5]:
    print(key, value)

8019
(40.79, -73.795) {(40.79, -73.792): 0.009400790831058076, (40.79, -73.804): 0.02665183346506693, (40.79, -73.786): 0.02624477279791465, (40.79, -73.796): 0.002410152439330088, (40.79, -73.794): 0.003181105704966562, (40.79, -73.795): 0.003844184044364148, (40.78, -73.795): 0.005251628822716274, (40.79, -73.791): 0.009741447042992716, (40.79, -73.797): 0.0051788418127648355}
(40.79, -73.792) {(40.79, -73.793): 0.0043429778407692915, (40.78, -73.792): 0.004001166562623688, (40.79, -73.795): 0.008208588715706403, (40.79, -73.79): 0.004667697540932431}
(40.79, -73.804) {(40.79, -73.806): 0.005715005993435982, (40.79, -73.803): 0.0025524819987512363, (40.79, -73.802): 0.00762419999813146, (40.79, -73.807): 0.007768996385895298, (40.79, -73.804): 0.00566910111313372, (40.79, -73.805): 0.0026056926050026033, (40.79, -73.795): 0.009585919041816167, (40.79, -73.808): 0.010157507241036918}
(40.79, -73.786) {(40.79, -73.787): 0.0024113065076097023, (40.79, -73.786): 0.008801337587094928, (40

In [262]:
for key, value in list(clusters.items())[:5]:
    print(key, value)
    print(type(key[0]))

(40.66, -73.934) ['42467330', '3534079545', '42479643', '42481987', '42485107', '42494695', '42497535', '42516395', '42514760', '4929411563']
<class 'float'>
(40.66, -73.933) ['42467331', '42485112', '42489551', '42514692', '42467065']
<class 'float'>
(40.66, -73.932) ['42467333', '42468550', '42472191', '42479645', '42485115', '42514764', '42494701', '42497547', '4749827004', '42522040', '4375241836', '4375241842']
<class 'float'>
(40.66, -73.931) ['42467334', '42468552', '42472195', '42494706', '42497552', '42520965', '42467069']
<class 'float'>
(40.66, -73.93) ['42467335', '42468557', '42494709', '42497556', '42503266', '42513385']
<class 'float'>


In [263]:
for key, value in list(adjacency.items())[:5]:
    print(key, value)
    print(type(key[0]))

(40.79, -73.795) {(40.79, -73.792): 0.009400790831058076, (40.79, -73.804): 0.02665183346506693, (40.79, -73.786): 0.02624477279791465, (40.79, -73.796): 0.002410152439330088, (40.79, -73.794): 0.003181105704966562, (40.79, -73.795): 0.003844184044364148, (40.78, -73.795): 0.005251628822716274, (40.79, -73.791): 0.009741447042992716, (40.79, -73.797): 0.0051788418127648355}
<class 'float'>
(40.79, -73.792) {(40.79, -73.793): 0.0043429778407692915, (40.78, -73.792): 0.004001166562623688, (40.79, -73.795): 0.008208588715706403, (40.79, -73.79): 0.004667697540932431}
<class 'float'>
(40.79, -73.804) {(40.79, -73.806): 0.005715005993435982, (40.79, -73.803): 0.0025524819987512363, (40.79, -73.802): 0.00762419999813146, (40.79, -73.807): 0.007768996385895298, (40.79, -73.804): 0.00566910111313372, (40.79, -73.805): 0.0026056926050026033, (40.79, -73.795): 0.009585919041816167, (40.79, -73.808): 0.010157507241036918}
<class 'float'>
(40.79, -73.786) {(40.79, -73.787): 0.0024113065076097023, 

In [ ]:
def dijkstras(graph, start, target):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == target:
            break

        if current_distance <= distances[current_node]:
            for neighbor, weight in graph[float(current_node)].items():
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))

    return distances[float(target)]

In [ ]:
for node in list(nodes.items())[:5]:
    print(node)

In [ ]:
coordinates_dict = {}

for node_id, coordinates in nodes.items():
    lon = coordinates['lon']
    lat = coordinates['lat']
    key = (lon, lat)
    coordinates_dict[key] = node_id

for node in list(coordinates_dict.items())[:5]:
    print(node)

In [ ]:
def find_closest_coordinates(target_coordinates, known_coordinates):
    min_distance = float('inf')
    closest_coordinates = None

    for key, value in known_coordinates.items():
        distance = euclidean_distance(target_coordinates[0], target_coordinates[1], value['lat'], value['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_coordinates = key

    return closest_coordinates


In [ ]:

def match_passenger_to_driver_t3(drivers, passengers, adjacency_dict, nodes):
    start_time = time.time()

    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        shortest_path_time = float('inf')
        current_passenger = None
        current_passenger_time = None

        for passenger_time, passenger in passenger_heap:
            if passenger_time < current_driver_time:
                #print((float(current_driver[1]), float(current_driver[2])))
                # Get the node IDs from the latitude and longitude using the Nodes dictionary
                start_node = find_closest_coordinates((float(current_driver[1]), float(current_driver[2])),nodes)
                end_node = find_closest_coordinates((float(passenger[1]), float(passenger[2])),nodes)

                path_time = dijkstras(adjacency_dict, start_node, end_node)

                if path_time < shortest_path_time:
                    shortest_path_time = path_time
                    current_passenger = passenger
                    current_passenger_time = passenger_time

        if current_passenger:
            print(current_passenger)
            passenger_heap.remove((current_passenger_time,current_passenger))
            current_driver[1] = current_passenger[3]
            current_driver[2] = current_passenger[4]
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")

            # Introduce a 95% chance for the driver to be pushed back
        if random.random() < 0.2:
            drive_duration = timedelta(hours=shortest_path_time)
            heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")

# Example usage
# Assuming you have an adjacency_dict, drivers_matrix, passengers_matrix, and Nodes dictionary
match_passenger_to_driver_t3(drivers_matrix[1:], passengers_matrix[1:], adjacency, nodes)


In [275]:
# Create clusters graph 
def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 2)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)
print(clusters)
def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 2))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 2))

        #print(cluster_key1,cluster_key2)
        # if the cluster keys are the same, still add it to the graph. 
        # if they are different, add it to the graph as an edge

        length = attributes["length"]
        speeds = attributes['weekday_hours']
        average_speed = sum(speeds)/len(speeds)
        estimated_time = length / average_speed
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(estimated_time)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [estimated_time]
      
        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency = update_edges_between_clusters(clusters, nodes)

for key1, nested_dict in adjacency.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            avg_list = sum(value) / len(value) 
            
            # Update the nested dictionary with the average list
            adjacency[key1][key2] = avg_list

print(len(adjacency))
for key, value in list(adjacency.items())[:5]:
    print(key, value)

{(40.66, -73.93): ['42467330', '42467331', '42467333', '42467334', '42467335', '42467337', '42467339', '42467343', '42467346', '42467776', '42467777', '42468550', '42468552', '42468557', '42468562', '42468566', '42468569', '42468570', '42468573', '42468574', '42468576', '42468577', '3534079545', '42472185', '42472191', '42472195', '42473144', '42473147', '42479643', '42479645', '42479646', '42481987', '42485107', '42485112', '42485115', '42489551', '42514764', '42494693', '42494695', '42494701', '42494706', '42494709', '42494712', '42494716', '42494719', '42497488', '42497490', '42497493', '42497496', '42497535', '42497547', '42497552', '42497556', '42497560', '42516395', '42500152', '42500183', '42500184', '42503266', '4749827004', '42513375', '42513385', '42514692', '42514697', '42514702', '42514710', '42514760', '42497498', '42520965', '42522002', '42522034', '42522040', '42522051', '42522054', '4929411563', '4375241836', '4207995533', '4375241842', '3547815597', '3654919263', '4246

In [276]:
print(len(clusters))

949


In [277]:
def floyd_warshall(graph):
    vertices = list(graph.keys())

    # Initialize the distance matrix and the dictionary for intermediate vertices
    dist = []
    nIndex = {}
    
    # Create the nIndex using a regular for loop
    for index, key in enumerate(vertices):
        nIndex[key] = index
    
    # Initialize the distance matrix
    #counte = 0
    for i in vertices:
        row = []
        for j in vertices:
            if i == j:
                row.append(0)
            elif j in graph[i]:
                row.append(graph[i][j])
            else:
                row.append(float('inf'))
        dist.append(row)
        #counte = counte + 1
        #print(counte)

    # Rest of the Floyd-Warshall algorithm
    for k in vertices:
        print(f"flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR")
        for i in vertices:
            #print(f"MIDDLEEEEeeEEEEEE")
            for j in vertices:
                #print(f"inner loop")
                if dist[nIndex[i]][nIndex[k]] + dist[nIndex[k]][nIndex[j]] < dist[nIndex[i]][nIndex[j]]:
                    dist[nIndex[i]][nIndex[j]] = dist[nIndex[i]][nIndex[k]] + dist[nIndex[k]][nIndex[j]]

    return dist, nIndex


adjacency_mat = [[[]]]
print(f"at this step")
ftime = time.time()
adjacency1 , indices = floyd_warshall(adjacency) #floyd warshall using average speeds (for all days of the week between clusters)
et = time.time()
elapsed_time = et - ftime
print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
print(f"YAYYYYYY!!!!!!!")
adjacency_mat.append(adjacency1)
for i in range(20):
    print(adjacency_mat[0][1][i])



at this step
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR
flo

IndexError: list index out of range

In [311]:
def find_closest_coordinates(target_coordinates, known_coordinates):
    min_distance = float('inf')
    closest_coordinates = None

    for key, value in known_coordinates.items():
        distance = euclidean_distance(target_coordinates[0], target_coordinates[1], key[0], key[1])
        if distance < min_distance:
            min_distance = distance
            closest_coordinates = key

    return closest_coordinates

def match_passenger_to_driver_t4(drivers, passengers, adjacency_matrices, nodes, nIndex):
    start_time = time.time()

    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)

    count = 0

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        shortest_path_time = float('inf')
        current_passenger = None
        current_passenger_time = None

        for passenger_time, passenger in passenger_heap:
            minni = timedelta(minutes=7)
            max_wait_time = current_driver_time - minni

            if passenger_time < current_driver_time and passenger_time > max_wait_time:
                start_node = find_closest_coordinates((float(current_driver[1]), float(current_driver[2])), nodes)
                end_node = find_closest_coordinates((float(passenger[1]), float(passenger[2])), nodes)

                # Determine if the date is a weekday or weekend
                """ if passenger_time.weekday() < 5:
                    adjacency_matrix = adjacency_matrices[0]
                else:
                    adjacency_matrix = adjacency_matrices[1] """
                print(start_node)
                print(end_node)
                print(nIndex[start_node])
                print(nIndex[end_node])
                path_time = adjacency_matrices[nIndex[start_node]][nIndex[end_node]]
                
                if path_time < shortest_path_time:
                    shortest_path_time = path_time
                    current_passenger = passenger
                    current_passenger_time = passenger_time

        if current_passenger:
            print(current_passenger)
            passenger_heap.remove((current_passenger_time, current_passenger))
            current_driver[1] = current_passenger[3]
            current_driver[2] = current_passenger[4]
            count = count + 1
            print(count)
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")

            # Introduce a 95% chance for the driver to be pushed back
        if random.random() < 0.2:
            if shortest_path_time < 10000:
                drive_duration = timedelta(hours=shortest_path_time)
            else:
                drive_duration = timedelta(minutes=2)
            heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")

In [316]:
match_passenger_to_driver_t4(drivers_matrix[1:], passengers_matrix[1:], adjacency1, clusters, indices)

(40.67, -73.87)
(40.72, -74.01)
259
41
(40.67, -73.87)
(40.75, -74.0)
259
25
['04/25/2014 00:08:00', '40.7157', '-74.0064', '40.7064', '-74.0167']
1
Driver assigned to Passenger: ['04/25/2014 00:14:00', '40.7064', '-74.0167'] -> ['04/25/2014 00:08:00', '40.7157', '-74.0064', '40.7064', '-74.0167']
(40.65, -73.78)
(40.73, -74.01)
931
60
(40.65, -73.78)
(40.75, -73.97)
931
32
['04/25/2014 00:20:00', '40.7547', '-73.9737', '40.7347', '-73.9944']
2
Driver assigned to Passenger: ['04/25/2014 00:21:00', '40.7347', '-73.9944'] -> ['04/25/2014 00:20:00', '40.7547', '-73.9737', '40.7347', '-73.9944']
(40.76, -74.0)
(40.73, -74.0)
46
17
(40.76, -74.0)
(40.76, -73.98)
46
85
(40.76, -74.0)
(40.73, -73.99)
46
18
(40.76, -74.0)
(40.72, -74.01)
46
41
['04/25/2014 00:29:00', '40.7554', '-73.984', '40.7236', '-74.0059']
3
Driver assigned to Passenger: ['04/25/2014 00:31:00', '40.7236', '-74.0059'] -> ['04/25/2014 00:29:00', '40.7554', '-73.984', '40.7236', '-74.0059']
(40.74, -74.0)
(40.75, -73.99)
73
